In [1]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
from torch.nn import CrossEntropyLoss
import numpy as np

/home/minigpt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/minigpt_env/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/minigpt_env/lib/libcudart.so.11.0'), PosixPath('/home/minigpt_env/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [2]:
#torch.cuda.empty_cache()

In [3]:
path="/home/vicuna-weights-7B"
tokenizer = LlamaTokenizer.from_pretrained(path)
tokenizer.pad_token_id = ( 0)
tokenizer.padding_side = "left"

In [4]:
block_size = 128
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/home/medical-llama/finetune-data/medical-train.txt",
    block_size=block_size,)

/home/minigpt_env/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
block_size = 128
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/home/medical-llama/finetune-data/medical-val.txt",
    block_size=block_size,)

In [6]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
) 

In [7]:
torch.set_default_dtype(torch.half)
model = LlamaForCausalLM.from_pretrained("/home/vicuna-weights-7B")
torch.set_default_dtype(torch.float)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.73s/it]


In [8]:
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
 inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1,)
model = get_peft_model(model, peft_config).to('cuda:0')
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742618112 || trainable%: 0.06220586618327525


In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    ignore_index = 0
    loss_fct = CrossEntropyLoss(ignore_index=ignore_index)
    logits = logits.view(-1, logits.shape[-1])
    labels = labels.view(-1)
    loss = loss_fct(logits, labels)
    perplexity = np.exp(loss.item())
    return {"perplexity": perplexity}

In [10]:
training_args = TrainingArguments(
    
    output_dir="~/medical-llama/output-vicuna-finetune",
    overwrite_output_dir=True,
    logging_dir="~/medical-llama/logs-vicuna-finetune",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    save_steps=200,
    save_total_limit=3,
    fp16=True,
    optim="adamw_torch",
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    learning_rate=0.00005,
    logging_steps=20,
    evaluation_strategy='steps',
    eval_steps=40,
    
    
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
     
)

In [12]:
model = torch.compile(model)

In [13]:
#torch.cuda.empty_cache()

In [14]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: deelipvenkat. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


TrainOutput(global_step=5055, training_loss=2.2372131423120094, metrics={'train_runtime': 27147.8467, 'train_samples_per_second': 23.844, 'train_steps_per_second': 0.186, 'total_flos': 3.285441732824531e+18, 'train_loss': 2.2372131423120094, 'epoch': 3.0})

In [15]:
model.save_pretrained("/home/medical-llama/vicuna-lora-7B")

In [16]:
trainer.save_model("/home/medical-llama/vicuna-lora-7B-full")
